In [ ]:
import pandas as pd
from datetime import datetime
df_lists = pd.read_csv("C:/python/python_for_data_science/hackason/listings.csv")
df_reviews = pd.read_csv("C:/python/python_for_data_science/hackason/reviews.csv.gz")
df_calender = pd.read_csv("C:/python/python_for_data_science/hackason/calendar.csv.gz")

In [42]:
# in these columns, there are all NaN data. We deleted them.
df_calender.drop(columns=["price", "adjusted_price"], inplace = True)

In [43]:
import pandas as pd

# merged all reviews on the same listing_id
reviews_text = df_reviews.groupby("listing_id")["comments"].apply(
    lambda x: " ".join(x.dropna().astype(str))
).reset_index()

reviews_text.rename(columns={"comments":"all_comments"}, inplace=True)

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=500, stop_words="english")

#vectorized reviews
tfidf_matrix = tfidf.fit_transform(reviews_text["all_comments"].fillna(""))

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(),
                        columns=[f"tfidf_{w}" for w in tfidf.get_feature_names_out()])

# added listing_id
tfidf_df["listing_id"] = reviews_text["listing_id"]

In [45]:
merged = pd.merge(df_calender, df_lists, left_on="listing_id", right_on="id", how="left")
merged_2 = pd.merge(merged, tfidf_df, left_on="listing_id", right_on="listing_id", how="left")

In [46]:
merged_2.head()

,listing_id,date,available,minimum_nights_x,maximum_nights,id,name,host_id,host_name,neighbourhood_group,...,tfidf_wszystko,tfidf_zona,tfidf_zu,tfidf_zum,tfidf_équipé,tfidf_était,tfidf_été,tfidf_все,tfidf_на,tfidf_очень
0,15526,2025-06-28,f,5,1125,15526,"Classy apartment ""Palazzo""",60754,Maurizio,NaN,...,0.0,0.0,0.151539,0.088017,0.0302,0.013319,0.0,0.0,0.0,0.0
1,15526,2025-06-29,t,5,1125,15526,"Classy apartment ""Palazzo""",60754,Maurizio,NaN,...,0.0,0.0,0.151539,0.088017,0.0302,0.013319,0.0,0.0,0.0,0.0
2,15526,2025-06-30,t,5,1125,15526,"Classy apartment ""Palazzo""",60754,Maurizio,NaN,...,0.0,0.0,0.151539,0.088017,0.0302,0.013319,0.0,0.0,0.0,0.0
3,15526,2025-07-01,t,5,1125,15526,"Classy apartment ""Palazzo""",60754,Maurizio,NaN,...,0.0,0.0,0.151539,0.088017,0.0302,0.013319,0.0,0.0,0.0,0.0
4,15526,2025-07-02,t,5,1125,15526,"Classy apartment ""Palazzo""",60754,Maurizio,NaN,...,0.0,0.0,0.151539,0.088017,0.0302,0.013319,0.0,0.0,0.0,0.0


In [ ]:
merged_2.columns.tolist()